In [1]:
import os

In [2]:
%pwd

'/Users/rupeshyadav/Documents/MLFLOW/End-To-End-MLOPs-Credit_Card_Fraud_Detection/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rupeshyadav/Documents/MLFLOW/End-To-End-MLOPs-Credit_Card_Fraud_Detection'